# MZA with 6 strain rates

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("mime")
import pandas as pd
import h5py
from scipy.optimize import curve_fit

colors = ['#bb0000', '#00bb00', "#0000bb", '#bbbb00', '#bb00bb', "#00bbbb", '#bbbbbb', '#770000', '#007700', "#000077", '#777700', '#770077', "#007777", '#777777', '#440000', '#004400', "#000044", '#444400', '#440044', "#0044444", '#444444','#000000']

In [ ]:
baseSize = (8, 6)  # Base size of a subplot

def sbPlot(n):
    if (n == 1): return 1, 1
    if (n <= 2): return 1, 2
    if (n <= 4): return 2, 2
    if (n <= 6): return 3, 2
    if (n <= 9): return 3, 3
    if (n <= 12): return 4, 3
    return 0, 0

def sbPlotSize(n):
    x, y = sbPlot(n)
    return baseSize[0] * y, baseSize[1] * x

In [ ]:
h5f = h5py.File('../GleebleData.h5','r')
allData = h5f['all'][:]
shortData = h5f['short'][:]
h5f.close()

In [ ]:
allData.shape, shortData.shape

Remove first point of each curve, where $\varepsilon^p=0$

In [ ]:
allData = allData[allData[:,0] != 0]
shortData = shortData[shortData[:,0]!=0]
identData = allData[allData[:,0] > 0.01]

In [ ]:
strains = np.unique(identData[:,0])
allStrains = np.unique(allData[:,0])
epsps = np.unique(identData[:,1])
temperatures = np.unique(identData[:,2])
nEps = len(strains)
nEpsp = len(epsps)
nTemp = len(temperatures)
strains, epsps, temperatures, nEps, nEpsp, nTemp

# Identification of the MZA parameters
$\sigma^y = (C_1 + C_2\varepsilon^n) \exp\left[-(C_3 + C_4\varepsilon^p) + (C_5 + C_6(T-T_0))\ln\left(\frac{\dot{\varepsilon^p}}{\dot{\varepsilon^p_0}}\right)\right]$

In [ ]:
T0 = temperatures[0]
epsp0 = epsps[0]
Tm = 1460

In [ ]:
C1 = allData[(allData[:,1]==epsp0) & (allData[:,2]==T0)][0,3]
print("C1=%g" %(C1))

Get the curves for the reference strain rate

In [ ]:
Sig = identData[identData[:,1] == epsp0]

In [ ]:
I1 = []
S1 = []
J1 = []
for epsj in strains:
    sbdata = Sig[Sig[:,0] == epsj]
    params = np.polyfit((sbdata[:,2] - T0), np.log(sbdata[:,3]), 1)
    s1, i1 = params
    j1 = np.exp(i1)
    S1.append(s1)
    I1.append(i1)
    J1.append(j1)

In [ ]:
(J1-C1)[2:]

In [ ]:
params = np.polyfit(np.log(strains), np.log(J1-C1), 1)
s, i = params
n = s
C2 = np.exp(i)
print("n=%g, C2=%g" %(n, C2))

In [ ]:
params = np.polyfit(strains, S1, 1)
s, i = params
C3 = -i
C4 = -s
print("C3=%g, C4=%g" %(C3, C4))

In [ ]:
def LinearFunction(x, a, b):
    return a*x + b

c5 = []
c6 = []
cl=0
for epsj in strains[:10]:
    sbdata = identData[identData[:,0]==epsj]
    S2 = []
    for temp in temperatures:
        sbdata1 = sbdata[sbdata[:,2]==temp]
        params1 = np.polyfit(np.log(sbdata1[:,1]/epsp0), (np.log(sbdata1[:,3])), 1)
        s1, i1 = params1
        S2.append(s1)
    params2 = np.polyfit(temperatures-T0, S2, 1) 
    s2,i2 = params2
    s2,i2
    c5.append(i2)
    c6.append(s2)
    cl +=1
C5 = np.mean(c5)
C6 = np.mean(c6)
C5,C6
print("C5=%g, C6=%g" %(C5, C6))

In [ ]:
C1,C2,C3,C4,C5,C6,n

In [ ]:
def MZAconstitutiveLaw(eps, epsp, T):
    return (C1 + C2 * eps**n) * np.exp(-(C3 + C4 *eps) *(T-T0) + (C5 + C6*(T-T0))*np.log(epsp / epsp0))

In [ ]:
# Plot the curves
from matplotlib.lines import Line2D
def create_dummy_line(**kwds):
    return Line2D([], [], **kwds)

plt.figure(figsize = sbPlotSize(nEpsp))
plt.rc('text', usetex = True)
idx = 1
plt.subplots_adjust(hspace = 0.3)
for epsp in list(epsps):
    legendLine = []
    xs, ys = sbPlot(nEpsp)
    plt.subplot(xs, ys, idx)
    sbdata = shortData[shortData[:,1]==epsp]
    cl =0
    for temp in list(temperatures):
        sbdata1 = sbdata[sbdata[:,2]==temp]
        plt.plot(sbdata1[:,0], sbdata1[:,3], colors[cl], marker = 's', markersize = 5, linestyle = 'none')
        plt.plot(allStrains, MZAconstitutiveLaw(allStrains, epsp, temp), colors[cl], linewidth = 2.5)
        plt.rcParams['xtick.labelsize'] = 16
        plt.rcParams['ytick.labelsize'] = 16
        legendLine.append((r'$T=$' + str(int(temp)) + r'$^{\circ}$C', {'color':colors[cl], 'linestyle':'-', 'linewidth':2.5, 'marker':'s'}))
        cl +=1
    plt.xlim(0, 0.7)
    plt.ylim(bottom=0)
    plt.xlabel(r'plastic strain $\varepsilon^p$', fontsize = 16) # Labels the x axis
    plt.ylabel(r'flow stress $\sigma^y$ (MPa)', fontsize = 16) # Labels the y axis
    plt.title(r'$\dot{\varepsilon^p} = ' + str(epsp) + '$ s$^{-1}$', fontsize = 16) # Self explicit command
    idx += 1
plt.legend([create_dummy_line(**l[1]) for l in legendLine],[l[0] for l in legendLine], 
           loc = 'upper center', fontsize = 12, ncols = 5, bbox_to_anchor = (0.0, -0.2), shadow = False)
plt.savefig("CompExpMZA.svg")
plt.show()

In [ ]:
MZAstress = MZAconstitutiveLaw(allData[:,0], allData[:,1], allData[:,2])

In [ ]:
data = np.concatenate((allData[:,0:3],MZAstress.reshape((MZAstress.shape[0],1))),axis=1)
h5f = h5py.File('MZA-6.h5','w')
h5f.create_dataset('data', data = data)
h5f.close()

In [ ]:
EAAR = np.sum(np.abs((allData[:,3] - MZAstress)/(allData[:,3])))*100/MZAstress.shape[0]
print("EAAR = %.2f" %(EAAR) + ' %')

In [ ]:
RMSE = np.sqrt(np.sum((allData[:,3] - MZAstress)**2)/MZAstress.shape[0])
print('RMSE = %.2f' %(RMSE)+' MPa')

In [ ]:
C1,C2,C3,C4,C5,C6,n

In [ ]:
def outOf(val, i):
    if ((val<-i) or (val>i)): return True
    return False

def conv (v, d):
    va = abs(v)
    e = int(np.log10(va))
    if (va < 1): e-=1
    if outOf(e,2):
        a = v * 10**(-e)
        v = np.round(a * 10**d)/10**d
        return str(v)+'\\times 10^{'+str(e)+'}'
    v = np.round(v * 10**d)/10**d
    return str(v)

print("C1 =",conv(C1,4))
print("C2 =",conv(C2,4))
print("C3 =",conv(C3,4))
print("C4 =",conv(C4,4))
print("C5 =",conv(C5,4))
print("C6 =",conv(C6,4))
print("n =",conv(n,4))

print(conv(C1,4),'&',conv(C2,4),'&',conv(C3,4),'&',conv(C4,4),'&',conv(C5,4),'&',conv(C6,4),'&',conv(n,4))